In [ ]:
!pip install peakutils

In [ ]:
from peakutils import peak
import pandas as pd
import numpy as np
import cv2
from scipy.spatial import distance
import matplotlib.pyplot as plt
import pydicom

In [ ]:
df = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
df

In [ ]:
class MelanomaABCDE():
    def __init__(self,
#         threshold = [127, 255, 0],
        resolution = 64,
        scoreThreshold = 0.0,
    ):
        super().__init__()
#         self.threshold = threshold
        self.resolution = resolution
        self.scoreThreshold = scoreThreshold
        self.contours = None
        self.center = None
        self.radius = None
        self.img = None
        self.preproimg = None

        
# =======================================================
# =================PREPROCESSING=========================
# =======================================================


    def getThresholds(self, gray):
        unique, counts = np.unique(gray, return_counts=True)
        slopes = counts[1:] - counts[:-1]
        mode = unique[np.argmax(counts)]
        minlim = slopes[:mode][slopes[:mode]>2250]
        if len(minlim) == 0:
            return np.argmax(slopes)
        threshlim = minlim[minlim<2500]
        if len(threshlim) == 0:
            threshlim = np.min(minlim)
            return np.where(slopes==threshlim)[0][0]
        else:
            return np.where(slopes==threshlim[0])[0][0]
        
    def preprocessImage(self, img):
        img = cv2.medianBlur(img, 11)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        ptsx = np.sum(gray, axis=0, dtype=np.int32)
        peaksx = peak.indexes(ptsx, thres=1/ptsx, min_dist=1000)
        ptsy = np.sum(gray, axis=1, dtype=np.int32)
        peaksy = peak.indexes(ptsy, thres=1/ptsy, min_dist=1000)
        if(len(peaksx)>1):
            if(len(peaksy)>1):
                gray = gray[peaksy[0]:peaksy[-1], peaksx[0]:peaksx[-1]]
                img = img[peaksy[0]:peaksy[-1], peaksx[0]:peaksx[-1],:]
            else:
                gray = gray[:, peaksx[0]:peaksx[-1]]
                img = img[:, peaksx[0]:peaksx[-1],:]
        elif(len(peaksy)>1):
            gray = gray[peaksy[0]:peaksy[-1],:]
            img = img[peaksy[0]:peaksy[-1],:,:]

        img = self.hair_remove(img)            
        (height, width) = gray.shape
        self.img = img
        threshold = [0]*3
        threshold[1] = self.getThresholds(gray)
        
        if threshold[2] == 1:
            ret,thresh = cv2.threshold(gray, threshold[0], threshold[1],
                                       cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        else:
            ret,thresh = cv2.threshold(gray, threshold[0], threshold[1],
                                        cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        return thresh

    def getCentroids(self, img):
        cnts = cv2.findContours(img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts_len = list(map(len, cnts[0]))
        cnts = cnts[0][cnts_len.index(max(cnts_len))]
        M = cv2.moments(cnts)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])   
        radii = list(map(distance.euclidean, cnts, [[cX, cY]]*len(cnts)))
        max_Radius = np.round(max(radii))
        max_Radius = np.min([img.shape[0]-cY, img.shape[1]-cX, max_Radius, cX, cY])
        return [(cX, cY), max_Radius]

    
    def hair_remove(self, image):
        shape = image.shape[:-1]
        image = cv2.resize(image, (1024, 1024))
        grayScale = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        kernel = cv2.getStructuringElement(1,(17,17))
        blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
        _,threshold = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)
        final_image = cv2.inpaint(image,threshold,1,cv2.INPAINT_TELEA)
        return cv2.resize(final_image, (shape[1], shape[0]))

# =======================================================
# =================BORDER SHAPE==========================
# =======================================================


    def area_ratio(self, img=None, max_radius=None):
        cX, cY = max_radius, max_radius
        blob_area = 0
        for (i, row) in enumerate(img):
            for (j, pix) in enumerate(row):
#                 print(i,j)
                if((i-cY)**2 + (j-cX)**2 <= (max_radius**2) and pix>0):
                    blob_area+=1

        area = np.pi*max_radius*max_radius
        return blob_area/area
 
    
# =======================================================
# =================SYMMETRY==============================
# =======================================================


    def score(self, img):
        img = np.float32(img)
        G_X = cv2.reduce(img, 0, cv2.REDUCE_SUM, cv2.CV_32F)
        G_Y = cv2.reduce(img, 1, cv2.REDUCE_SUM, cv2.CV_32F)
        return cv2.compareHist(G_X, G_Y.transpose(), cv2.HISTCMP_CORREL)
    
    def rotateImage(self, image, angle):
        (h, w) = image.shape[:2]

        center = (w / 2, h / 2)

        M = cv2.getRotationMatrix2D(center, angle, 1.44)
        rotated = cv2.warpAffine(image, M, (w, h))
        return rotated
    
    def detectSymmetry(self, img):
        img = self.preproimg
        (cx, cy) = self.center
        max_radius = self.radius
        cropped_img = img[cy-max_radius:cy+max_radius, cx-max_radius:cx+max_radius]
        Scores = np.zeros((self.resolution, 2))
        for i in range(0, 180, 180//self.resolution):
            indx = i*self.resolution//180
            rotated_img = self.rotateImage(cropped_img, i)
            Scores[indx][0] = i-45
            Scores[indx][1] = self.score(rotated_img)
        return Scores[np.where(Scores[:,1]>self.scoreThreshold)]

    def getSymmetry(self, img, angle):
        img = self.preproimg
        (cx, cy) = self.center
        max_radius = self.radius
        cropped_img = img[cy-max_radius:cy+max_radius, cx-max_radius:cx+max_radius]
        angle += 45
        rotated_img = self.rotateImage(cropped_img, angle)
        return self.score(rotated_img)

    def getMaxSymmetry(self, img):
        output = self.detectSymmetry(img)
        max_symmetry = output[np.where(output[:,1] == max(output[:,1]))]
        return (max_symmetry.tolist()[0])

    
# =======================================================
# =====================COLOR=============================
# =======================================================

    def color_consistancy(self, img):
        (cx, cy) = self.center
        max_radius = self.radius
        cropped_img = self.img[cy-max_radius:cy+max_radius, cx-max_radius:cx+max_radius]
        cropped_thresh = self.preproimg[cy-max_radius:cy+max_radius, cx-max_radius:cx+max_radius]
        masked = cv2.bitwise_and(cropped_img, cropped_img, mask=cropped_thresh)
        red, redcount = np.unique(masked[:,:,0].ravel(), return_counts=True)
        green, greencount = np.unique(masked[:,:,1].ravel(), return_counts=True)
        blue, bluecount = np.unique(masked[:,:,2].ravel(), return_counts=True)
        return {
            'redmean' : np.mean(np.repeat(red[1:], redcount[1:])),
            'greenmean' : np.mean(np.repeat(green[1:], greencount[1:])),
            'bluemean' : np.mean(np.repeat(blue[1:], bluecount[1:])),
            'redstd' : np.std(np.repeat(red[1:], redcount[1:])),
            'greenstd' : np.std(np.repeat(green[1:], greencount[1:])),
            'bluestd' : np.std(np.repeat(blue[1:], bluecount[1:]))

        }

    
    
    
# =======================================================
# =====================MAIN==============================
# =======================================================
    
    
    def main(self, img):
        try:
            self.img = img
            preproimg = self.preprocessImage(img)
            self.preproimg = preproimg

            [(cX, cY), max_radius] = self.getCentroids(preproimg)
            self.center = (cX,cY)
            max_radius = int(max_radius)
            self.radius = max_radius

            cropped_img = self.img[cY-max_radius:cY+max_radius, cX-max_radius:cX+max_radius]
            cropped_thresh = self.preproimg[cY-max_radius:cY+max_radius, cX-max_radius:cX+max_radius]

            symmetry = self.getMaxSymmetry(preproimg)
            area_score = self.area_ratio(cropped_thresh, max_radius)
            color_score = self.color_consistancy(self.img)

            self.contours = None
            self.center = None
            self.radius = None
            self.img = None
            self.preproimg = None
            return np.array([symmetry[1], area_score, 
                             color_score['redmean'], color_score['greenmean'], color_score['bluemean'], 
                             color_score['redstd'], color_score['greenstd'], color_score['bluestd']])
        except:
            return np.array([[np.nan]*8])
#         return area_score
#         return symmetry
        

In [ ]:
detector = MelanomaABCDE()

In [ ]:
df

In [ ]:
def feature_extraction(imgid):
    img = cv2.imread(f'../input/siim-isic-melanoma-classification/jpeg/train/{imgid}.jpg')
    output = detector.main(img)
    print(imgid)
    return output

In [ ]:
op = df.iloc[:500,0].apply(feature_extraction)

In [ ]:
pd.DataFrame(op).to_csv('op5.csv')

In [ ]:
op = df.iloc[500:1000,0].apply(feature_extraction)

In [ ]:
pd.DataFrame(op).to_csv('op10.csv')